In [ ]:
# 라이브러리 설치 
# !pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# 토큰화가 된 데이터 셋을 임의로 생성 
sentences = [
    ['오늘', '날씨', '좋다'], 
    ['강아지', '산책', '기분', '좋다'], 
    ['비', '오다', '산책', '못하다']
]

In [ ]:
# Word2Vec 객체를 생성하면서 데이터를 입력 
model = Word2Vec(
    sentences= sentences, 
    vector_size= 100, 
    window=5, 
    min_count=1, 
    sg= 1, 
    epochs= 100, 
    seed = 42 
)

In [ ]:
# 단위 벡터 출력 
print(model.wv['산책'])

In [ ]:
# 코사인 유사도 
print(model.wv.most_similar('산책'))

### Word2Vec 
- 문자를 수치형으로 변환 시켜주는 딥러닝 기반의 임베딩 기술

- 주요 매개변수 
    - sentences
        - 기본값 : None
        - 토큰화가 된 문장 데이터 ( 2차원 리스트의 형태 )
        - 문장의 데이터가 None 기본값이라면 -> 학습을 하는 메서드가 존재할것이다. 
    - vector_size
        - 기본값 : 100
        - 임베딩 백터 차원의 개수 (feature 수)
    - window
        - 기본값 : 5
        - 예측 시 고려할 주변 단어와의 거리 (문맥의 크기)
    - sg
        - 기본값 : 0
        - 0 인 경우 CBOW 방식(주변 단어를 이용하여 중심 단어를 예측)
        - 1 인 경우 Skip-Gram 방식(중심 단어를 이용하여 주변 단어를 예측)
        - 빠른 계산이 필요한 경우라면 0
        - 일반적인 경우에는 1은 선택
    - min_count
        - 기본값 : 5
        - 최소 등장 빈도 수 
        - 적게 등장하는 단어들을 제외
    - hs 
        - 기본값 : 0
        - 계산 횟수 지정 
        - 0 : Negative Sampling (계산량 적음)
        - 1 : Hierarchical Softmax ( 계산량 많음 )
    - epochs
        - 기본값 : 100
        - 반복 학습하는 횟수 지정 
    - max_vocab_size
        - 기본값 : None
        - 메모리 제한 시 사용할 최대 단어의 개수 
- 주요 속성 
    - wv
        - 학습된 단어 백터( 객체 형으로 출력 )
        - 예시 :  model.wv['단어']
    - wv.index_to_key
        - 단어의 리스트(학습이 된 단어의 개수 (최소 등장 횟수에 영향))
        - 등장 빈도수에 따라 자동 정렬 
    - wv.key_to_index
        - 단어 -> 인덱스로 매칭 
        - 특정 단어가 인덱스 몇에 위치하는가
    - copus_total_word
        - 전체 학습이 된 단어의 개수 
    - epochs
        - 학습 epoch의 수 
    - vector_size
        - 백터 차원의 수 
- 메서드 
    - wv.most_similar( {word}, topn = 10 )
        - 특정 단어와 유사한 단어를 출력 
        - topn은 유사한 단어의 개수 
    - wv.similarity({word1}, {word2})
        - 두 단어 간의 코사인 유사도 
    - wv.get_vector( {word} )
        - 특정 단어의 벡터를 반환
    - train( ... )
        - 추가 데이터로 학습 
    - save( {path} )
        - 모델 자체를 파일로 저장 
    - Word2Vec.load()
        - 모델을 불러오기 

In [ ]:
# 토큰화 (konlpy)
# 백터화 (Word2Vec)
# 입력데이터가 생성이 되었으니 머신러닝 모델에 학습 

In [ ]:
import numpy as np 
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Komoran

In [ ]:
X_raw = [
    '오늘 날씨가 좋다 여행 가고 싶다',
    '기온이 너무 떨어져서 아무것도 하기 싫다', 
    '음식이 정말 맛있었고 서비스도 좋았다', 
    '완전 별로고 돈이 아깝다 다시는 안 온다', 
    '수업이 너무 지루하고 졸리다', 
    '영화가 너무 재미있어서 시간이 가는 줄 몰랐다'
]
target = [1, 0, 1, 0, 0, 1]

In [ ]:
# 토큰화 작업 
komoran = Komoran()
# 사용할 형태들을 지정 
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']

tokens = []
for raw in X_raw:
    words = []
    for word, pos in komoran.pos(raw):
        if pos in allow_pos:
            words.append(word)
    tokens.append(words)

tokens

In [ ]:
# Word2Vec을 이용하여 학습(Skip-gram 방식)
w2v = Word2Vec(
    sentences= tokens, 
    vector_size= 100, 
    window = 5, 
    min_count= 1, 
    sg = 1, 
    epochs= 100, 
    seed = 42, 
    workers=2
)

In [ ]:
# w2v 객체에서 wv 속성을 자주 사용하기에 변수에 따로 저장 
wv = w2v.wv

In [ ]:
# tokens의 단어들 중  w2v의 index_to_key에 존재하는 데이터의 단위 백터를 확인 
vectors = []
for token in tokens:
    for word in token:
        # tokens의 단어들 중 Word2vec에서 학습 단어들에 포함되어있다면 
        vec = []
        if word in wv.index_to_key:
            # 해당 단어의 단어 백터를 출력하여 vertors의 대입 
            vec.append(
                wv[word]
            )
    vectors.append(np.mean(vec, axis=0))
vectors

In [ ]:
np.array(vectors).shape

In [ ]:
# vectors의 데이터의 의미 -> 독립 변수 
# target 데이터 -> 종속 변수 
svc = SVC(random_state=42)

In [ ]:
svc.fit(vectors, target)

In [ ]:
# 토큰화 함수 생성 
# komoran 사용 (konlpy 설치가 되어있는 경우)
# 설치가 되어있지 않은 경우에는 split()을 이용하여 토큰화 
def build_tokenize():
    try:
        # 라이브러리 로드 -> 라이브러리가 존재하면 코드들 실행 
        from konlpy.tag import Komoran
        komoran = Komoran()
        allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
        def tokenize(text):
            tokens = []
            for word, pos in komoran.pos(text):
                if pos in allow_pos:
                    tokens.append(word)
            return tokens
        # tokenize 함수를 결과로 되돌려준다. 
        return tokenize
    except Exception as e:
        print("Komoran 사용 불가 : ", e)
        return lambda x : x.split()


In [70]:
# build_tokenize 함수를 호출 
tokenize = build_tokenize()

In [ ]:
tokenize('오늘 날씨가 좋다')

In [ ]:
# 문장은 임베딩하는 함수를 생성 -> 벡터화 
# 단위 벡터의 평균을 구하는 함수 
def sent_embed_mean(tokens):
    vecs = []
    for word in tokens:
        if word in wv.index_to_key:
            vecs.append(wv[word])
    result = np.mean(vecs, axis=0) if vecs else np.zeros(wv.vector_size)
    return result

In [ ]:
# X_raw를 이용하여 벡터화 
# token 함수의 인자값은 하나의 문장 입력 
X_tokens = []
for raw in X_raw:
    X_tokens.append(
        tokenize(raw)
    )
X_tokens

In [ ]:
X_tokens2 = [ tokenize(raw) for raw in X_raw ]
X_tokens2

In [ ]:
# 임베딩 처리 
X_embed = []
for token in X_tokens:   #  ---- 2
    X_embed.append( 
        sent_embed_mean(token)   # --- 1
    )
X_embed

In [ ]:
X_embed2 = [ sent_embed_mean(token) for token in X_tokens ]
X_embed2

In [55]:
# X_embed -> 독립 변수 -> 문장을 토큰화하고 백터화 처리를 끝낸 데이터 
# target -> 종속 변수 -> 문장의 감정 표현 0 : 부정, 1 : 긍정
svc.fit(X_embed, target)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [60]:
# 데이터의 개수가 적당히 많은 수준인 경우 Train, Test 데이터를 나눠주고 
# Train 데이터를 이용하여 학습을 하고 Test 데이터를 이용해서 검증하는 함수 
svc = SVC(random_state=42)

def run_model(X, Y, test_size = 0.2):
    # X는 독립 변수
    # Y는 종속 변수
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size= test_size, random_state=42, stratify=Y
    )
    # 모델에 학습
    svc.fit(X_train, Y_train)
    # 학습된 모델에 예측 값
    y_pred = svc.predict(X_test)
    print("정확도 : ", round(
        accuracy_score(y_pred, Y_test), 4
    ))
    print('분류 레포드 : ')
    print(classification_report(y_pred, Y_test))


In [61]:
run_model(X_embed, target)

정확도 :  0.5
분류 레포드 : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67         2

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri

In [63]:
def func_1(*x, y):
    sum_x = sum(x)
    result = sum_x + y
    return result

func_1(1,2,3,4, y = 5)

15

In [64]:
def func_2(*, x, y):
    result = x *  y
    return result

In [66]:
func_2(x = 10, y = 3)

30

In [71]:
# 학습된 모델에 예측의 값을 반환하는 함수 
def predict_sentence_list(sentences, model):
    # sentences : 문장들의 리스트 
    # 문장들을 토큰화 -> 임베딩 
    X_test = []
    for sent in sentences:
        # token() 함수를 호출하여 토큰화 
        tokens = tokenize(sent)
        # 토큰화 된 문장을 sent_enbed_mean 함수에 입력하여 호출 ( 단위 백터의 평균 )
        vec = sent_embed_mean(tokens)
        X_test.append(vec)
    
    preds = model.predict(X_test)
    result = []
    for sent, pred in zip(sentences, preds):
        label = "긍정" if pred == 1 else "부정"
        result.append([sent, label])
    return result



In [68]:
new_sentences = [
    '영화가 너무 지루해서 돈이 아깝다', 
    '날씨가 너무 별로다', 
    '기온이 좋아서 어디론가 떠나고 싶다'
]

In [72]:
predict_sentence_list(new_sentences, svc)

[['영화가 너무 지루해서 돈이 아깝다', '부정'],
 ['날씨가 너무 별로다', '긍정'],
 ['기온이 좋아서 어디론가 떠나고 싶다', '긍정']]

In [74]:
# Word2Vec과 TF-IDF를 융합하여 임배딩 처리 함수 생성 
# 문맥 상에서 단어의 예측 벡터와 전체 문서에서 특정 단어들의 중요도를 결합한 벡터 데이터
# TF-IDF 백터화 행렬 생성 
tfidf_vec = TfidfVectorizer(
    tokenizer= tokenize, 
    lowercase= False
).fit(X_raw)
tfidf_vec

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,False
,preprocessor,None
,tokenizer,<function bui...002692E617F60>
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [76]:
idf = dict(
    zip(
        # get_feature_names_out() -> Tfidf에서 사용된 단어들의 목록
        tfidf_vec.get_feature_names_out(), 
        # idf_ : 중요도
        tfidf_vec.idf_
    )
)

In [78]:
# 단어 별 단위 벡터의 평균과 idf을 곱한다. 
def sent_embed_tfidf(tokens):
    vecs = []
    weight = []
    for word in tokens:
        # tokens에 각각의 단어가 Word2Vec과 TF-IDF에 존재한다면
        if word in wv.key_to_index and word in idf:
            # vecs -> 단위벡터와 중요도를 곱한 값을 vecs 추가
            vecs.append(wv[word] * idf[word])
            # weight -> 중요도 데이터를 추가 
            weight.append(idf[word])
    # vecs의 데이터가 존재하지 않는다면 -> tokens 안에 단어는 존재하지만 Word2Vec이나
    # TD-IDF에 단어가 존재하지 않을때
    if not vecs:
        # 희소 행렬 되돌려준다. 0행렬
        result = np.zeros(wv.vector_size)
    else:
        result = np.sum(vecs, axis=0) / ( np.sum(weight) + 1e-9 )
    return result


In [ ]:
# sent_embed_tfidf() -> 출력은 머신 러닝 모델에서 학습 데이터로 사용한 벡터 데이터 


In [79]:
# 학습된 모델에 예측의 값을 반환하는 함수 
# 세번째 매개변수(vec_type)를 생성 -> 기본값은 'mean'
# 'tfidf' 입력이 들어온다면 벡터화 작업은 w2v + ifidf 융합한 벡터화 
def predict_sentence_list(sentences, model, vec_type = 'mean'):
    # sentences : 문장들의 리스트 
    # 문장들을 토큰화 -> 임베딩 
    X_test = []
    for sent in sentences:
        # token() 함수를 호출하여 토큰화 
        tokens = tokenize(sent)
        # 토큰화 된 문장을 sent_enbed_mean 함수에 입력하여 호출 ( 단위 백터의 평균 )
        if vec_type == 'mean':
            vec = sent_embed_mean(tokens)
        elif vec_type == 'tfidf':
            vec = sent_embed_tfidf(tokens)
        X_test.append(vec)
    
    preds = model.predict(X_test)
    result = []
    for sent, pred in zip(sentences, preds):
        label = "긍정" if pred == 1 else "부정"
        result.append([sent, label])
    return result

